In [21]:
import pandas as pd

In [22]:
educ = pd.read_csv('educ.csv')

In [23]:
from bokeh.plotting import figure, show, output_notebook, output_file
from bokeh.models import Slider, Button, CheckboxGroup, CustomJS, ColumnDataSource, CDSView, Label
from bokeh.models.tools import HoverTool
from bokeh.models.filters import CustomJSFilter
from bokeh.models.widgets import Div
from bokeh.layouts import row, column, Spacer
from bokeh.resources import CDN
from bokeh.embed import file_html, json_item
#output_notebook()
output_file('educ.html',mode='cdn')

In [24]:
source = ColumnDataSource(educ)
fields = educ.FOE.unique().tolist()
N = len(fields)
m = 0.06
checkboxes = CheckboxGroup(labels=fields, active=[1,4,7,10],width=400)#,title='Broad fields of education')
p = figure(plot_width=1200, plot_height=800,x_range=(-m,1.15+m),y_range=(-m,1+m),toolbar_location='below',
          title='Market size against growth rate of educational disciplines')
p.title.text_font_style='bold'
p.title.text_font_size='16pt'
p.min_border_right=250
filter =  CustomJSFilter(code="""
let selected = checkboxes.active.map(i=>checkboxes.labels[i]);
let indices = [];
let column = source.data['FOE'];
for(let i=0; i<column.length; i++){
    if(selected.includes(column[i])){
        indices.push(i);
    }
}
return indices;
""", args=dict(checkboxes=checkboxes, source=source))

checkboxes.js_on_change("active", CustomJS(code="source.change.emit();", args=dict(source=source)))
qcolor = '#DBF3FA'
p.patch([0.5,1,1,0.75,0.75,0.5],[1,1,0.5,0.5,0.75,0.75],fill_color=qcolor,line_color=qcolor,alpha=0.8)
r = p.rect(x='X',y='Y',width_units='screen', height_units='screen',
           width=50,height=25, line_color='Colour',color='Colour',
           line_width=4,line_join='bevel',fill_alpha='alpha',source=source,view=CDSView(source=source, filters=[filter]))
scx = [0.125,0.375,0.625,0.875]*4              # x coordinates of square centres
scy = [0.125]*4+[0.375]*4+[0.625]*4+[0.875]*4  # y coordinates of square centres
g = p.rect(x=scx,y=scy,width_units='data', height_units='data',width=0.25,height=0.25,alpha=0)
t = p.text(x='X',y='Y',text='DisciplineAb',text_align='center',text_baseline='middle',
          source=source,view=CDSView(source=source, filters=[filter]))
p.add_tools(HoverTool(tooltips='@Discipline', renderers=[r]))

# The next lines make the labels below the x-axis
x_labels = ['SIZE:','Small','Medium','Large','Very large']
eftsl = ['EFTSL:','<5,000','5000 - 10000','10001 - 25,000','>25,000']
lx = [0, 0.125,0.375,0.625,0.875]
for i in range(5):
        p.add_layout(Label(x_units='data',y_units='data',x=lx[i],y=-0.03,text=x_labels[i],
                           text_align='center',text_font_style='bold'))
        p.add_layout(Label(x_units='data',y_units='data',x=lx[i],y=-0.06,text=eftsl[i],
                           text_align='center'))
        
# The next lines make the labels at the far right
ly = [0.125,0.375,0.625,0.875,1]
y_labels=['Negative','Low','Medium','High','GROWTH RATE']
y_labels2 = ['(CAGR 0%)','(CAGR 0-2%)','(CAGR 2-5%)','(CAGR>5%)','']
for i in range(5):
    p.add_layout(Label(x_units='data',y_units='data',x=1.01,y=ly[i]+0.02,text=y_labels[i],
                           text_align='left',text_font_style='bold'))
    p.add_layout(Label(x_units='data',y_units='data',x=1.01,y=ly[i]-0.02,text=y_labels2[i],
                           text_align='left'))
ticks = [0,0.25,0.5,0.75,1]
p.xaxis[0].ticker = []
p.yaxis[0].ticker = []
p.xgrid[0].ticker = ticks
p.ygrid[0].ticker = ticks
p.grid.grid_line_color='gray'
p.line([0,0],[-m,0],line_width=3,color='white')

div = Div(text="""<h2>Broad fields of education</h2>""",width=400, height=50)

div2 = Div(text="""<font-size="5">This chart shows the market sizes (measured by enrolments) and growth rates 
(measured by changes in enrolments) of a number of education disciplines.  This chart is interactive,
which means you can decide what to display, and also by mousing over the rectangle containing the abreviation 
you can see the full name of the discipline.</font-size>""",style={'font-size':'131%'},width=400, height=500)

show(row(column(div,checkboxes,Spacer(height=150),div2,width=400,height=800), p,width = 1200,height = 800,sizing_mode='fixed'))
output_file("educ.html",mode='inline')